In [ ]:

import gc

from tqdm import tqdm
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 120)

import datatable as dt
import dask.dataframe as dd

from copy import copy

from pathlib import Path


from time import time, sleep
from contextlib import contextmanager

import matplotlib.pyplot as plt

import plotly.express as plx
import seaborn as sns

import os, sys
import gc

from emulator import Iter_Valid

# print(plt.style.available)
# plt.style.use("dark_background")
# plt.style.use("seaborn-dark")


print(f"curdir {Path.cwd()}")

In [ ]:
dtypes = \
    {'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
#     'task_container_id': 'int16',
#     'user_answer': 'int8',
    'answered_correctly':'int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'}

In [ ]:
%%time


# valid_pd = pd.read_pickle("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid.pickle.zip").reset_index(drop=True)
# valid_pd = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather")  # .drop("row_id", axis=1)
train_pd = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train_1e5.feather", columns=dtypes.keys())  # .drop("row_id", axis=1)
# train_pd.to_parquet("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/train.parquet.gzip", compression="gzip")
# train_pd = dd.read_csv("/mnt/data30G/2020riid/train.csv", low_memory=False)  # .drop("row_id", axis=1)
# train_pd = pd.read_feather("/mnt/data30G/2020riid/riiid_train.feather")  # , columns=dtypes.keys())  # .drop("row_id", axis=1)

                   
# train_lectures_pd = train_pd.loc[train_pd.answered_correctly == -1]
# train_pd = train_pd.loc[train_pd.answered_correctly != -1]



In [ ]:
%%time

# for col, dtype in dtypes.items():
#     if col not in ["timestamp", "prior_question_elapsed_time"]:
#         train_dd[col] = train_dd[col].astype(dtype)

# train_pd.prior_question_had_explanation = train_pd.prior_question_had_explanation.fillna(False)
# train_pd.prior_question_elapsed_time = train_pd.prior_question_elapsed_time.fillna(0)

train_pd.astype(dtypes)

In [ ]:
%%time
# train_pd.groupby("user_id")[["timestamp", "prior_question_elapsed_time"]].agg(["mean", "min", "max", "count"])

In [ ]:
%%time
target = "answered_correctly"
# Answer for the previous questions of users
train_pd['lag'] = train_pd.groupby('user_id')[target].shift()
print(type(train_pd['lag']))
# For each user (groupby('user_id')), compute the cummulative number of correct answers and number answers in general
groupby = train_pd.groupby('user_id')['lag']
print(type(groupby))
cum = groupby.agg(['cumsum', 'cumcount'])

# User correctness (measure the users' learning progress)
train_pd['user_correctness'] = cum['cumsum'] / cum['cumcount']
# Drop the 'lag' feature
train_pd.drop(columns = ['lag'], inplace = True)

In [ ]:
train_dd.head()

In [ ]:
y_train = train_pd.pop("answered_correctly")

y_train.compute()


In [ ]:
users

In [ ]:
%%time

for col, dtype in dtypes.items():
    if col not in ["timestamp", "prior_question_elapsed_time"]:
        train_pd[col] = train_pd[col].astype(dtype, errors="ignore")
        gc.collect()
        # valid_pd[col] = valid_pd[col].astype(dtype, errors="ignore")

train_pd.timestamp = (train_pd.timestamp/(1000)).astype(np.int16, errors="ignore")
train_pd.prior_question_elapsed_time = (train_pd.prior_question_elapsed_time/1000).astype(np.int8, errors="ignore")

y_train = train_pd.pop("answered_correctly")
gc.collect()

In [ ]:
train_pd.head()

In [ ]:

train_pd = train_pd.loc[train_pd.content_type_id == 0]
# valid_pd = valid_pd.loc[valid_pd.content_type_id == 0]

# valid_pd = valid_pd.iloc[:int(1e4)]
# valid_pd.to_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather")

print("train_pd.shape", train_pd.shape)
# print("valid_pd.shape", valid_pd.shape)
train_pd.head()

In [ ]:

print(train_pd.timestamp.max(), train_pd.prior_question_elapsed_time.max())

In [ ]:
print(train_pd.timestamp.mean(), train_pd.prior_question_elapsed_time.mean())

In [ ]:
print(train_pd.timestamp.median(), train_pd.prior_question_elapsed_time.median())

In [ ]:
plt.figure(figsize=(15, 7))
train_pd.timestamp.hist(bins=50);

In [ ]:
print(f'Of the {train_pd.user_id.nunique()} users in train we have {train_pd[train_pd.timestamp == 0].user_id.nunique()} users with a timestamp zero row.')

In [ ]:
train_pd = train_pd.loc[train_pd.content_type_id == 0]
# valid_pd = valid_pd.loc[valid_pd.content_type_id == 0]

# valid_pd = valid_pd.iloc[:int(1e4)]
# valid_pd.to_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather")

print("train_pd.shape", train_pd.shape)
# print("valid_pd.shape", valid_pd.shape)
train_pd.head()

In [ ]:
train_pd.info()

## ПРОПУСКИ

### prior_question_had_explanation

In [ ]:
# в prior_question_had_explanation пропуски заполняем False
train_pd.prior_question_had_explanation = train_pd.prior_question_had_explanation.fillna(False)
train_pd.prior_question_had_explanation.value_counts(dropna=False)

### prior_question_elapsed_time

In [ ]:
# Попробую забить нулями, потом подумать
# TODO
train_pd.prior_question_elapsed_time = train_pd.prior_question_elapsed_time.fillna(0)


In [ ]:
train_pd.isna().any()

In [ ]:
%%time
# train_dd = dd.from_pandas(train_pd)
# users = train_dd.groupby("user_id")
train_dd = dd.read_parquet("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/train.parquet", low_memory=False, dtype=dtypes)
# train_dd = dd.read_csv("/mnt/data30G/2020riid/train.csv", low_memory=False)
users = train_dd.groupby("user_id").agg(['min', 'max']).compute()


In [ ]:
users

In [ ]:
train_pd = train_pd.loc[train_pd.content_type_id == 0]
# valid_pd = valid_pd.loc[valid_pd.content_type_id == 0]

# valid_pd = valid_pd.iloc[:int(1e4)]
# valid_pd.to_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather")

print("train_pd.shape", train_pd.shape)
# print("valid_pd.shape", valid_pd.shape)
train_pd.head()

In [ ]:
train_pd.isna().any()

In [ ]:
%%time
# train_dd = dd.from_pandas(train_pd)
# users = train_dd.groupby("user_id")
train_dd = dd.read_parquet("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/train.parquet", low_memory=False, dtype=dtypes)
# train_dd = dd.read_csv("/mnt/data30G/2020riid/train.csv", low_memory=False)
users = train_dd.groupby("user_id").agg(['min', 'max']).compute()


In [ ]:
users

In [ ]:
train_pd = train_pd.loc[train_pd.content_type_id == 0]
# valid_pd = valid_pd.loc[valid_pd.content_type_id == 0]

# valid_pd = valid_pd.iloc[:int(1e4)]
# valid_pd.to_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather")

print("train_pd.shape", train_pd.shape)
# print("valid_pd.shape", valid_pd.shape)
train_pd.head()

In [ ]:
train_pd.info()

## ПРОПУСКИ

### prior_question_had_explanation

In [ ]:
train_pd.prior_question_had_explanation.value_counts(dropna=False)

In [ ]:
len(train_pd.loc[train_pd.timestamp == 0])

In [ ]:
train_isna = train_pd[train_pd.prior_question_had_explanation.isna()]
train_pd = train_pd.loc[train_pd.prior_question_had_explanation.notna()]

print("len(train_pd[train_pd.timestamp == 0])", len(train_pd[train_pd.timestamp == 0]))
print("len(train_pd[train_pd.content_type_id == 1])", len(train_pd[train_pd.content_type_id == 1]))
print("train_pd.shape", train_pd.shape)
print("train_isna.shape", train_isna.shape)
train_pd.head()

In [ ]:
train_pd.user_id.nunique()

In [ ]:
train_pd.info()

In [ ]:
train_pd.memory_usage(deep=True)

In [ ]:
users = train_pd.groupby("user_id")
count = 0
count_zeros = 0
user_attempts = []
users_multiple_zero = []
for user_id, user_df in users:
    user_attempts.append(user_df.shape[0])
    zeros_timestamp = len(user_df.loc[user_df.timestamp == 0])
    if user_df.prior_question_had_explanation.isna().any():
        print(user_id)
    if zeros_timestamp > 1:
        count_zeros+=1
        users_multiple_zero.append(zeros_timestamp)
        print(count_zeros, user_id, zeros_timestamp)
    count+=1
print(count)
print("users_multiple_zero", users_multiple_zero)

In [ ]:
train_pd.loc[train_pd.timestamp == 0].user_id.unique()


In [ ]:
plt.figure(figsize=(20, 20))
sns.histplot(user_attempts)

In [ ]:
plx.histogram(user_attempts)

## Clustering


In [ ]:
validaten_flg = True
if validaten_flg:
    iter_test = Iter_Valid(valid_pd,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

## iterator
Now we can use iter_test(wrapper for env.iter_test) and set_predict(wrapper for env.predict) as usual.


In [ ]:



result = train_pd.copy()
users = train_pd.groupby("user_id")
count_ = users["index"].count()
print(type(count_))
max_ = users.max()
print(users)
# print(users)
result = result.merge(count_, left_on="user_id", right_index=True)
result = result.merge(max_, left_on="user_id", right_index=True)
result.head()
# result = result.merge()

In [ ]:
# %%time
#
# # pbar = tqdm(total=2500000)
# pbar = tqdm(total=valid_pd.shape[0])
# previous_test_df = None
# current_list = []
# for (current_test, current_prediction_df) in iter_test:
#     if previous_test_df is not None:
#         answers = eval(current_test["prior_group_answers_correct"].iloc[0])
#         responses = eval(current_test["prior_group_responses"].iloc[0])
#         previous_test_df['answered_correctly'] = answers
#         previous_test_df['user_answer'] = responses
#         # your feature extraction and model training code here
#
#     curtl =  len(current_test)
#     current_list.append(curtl)
#     previous_test_df = current_test.copy()
#     current_test = current_test[current_test.content_type_id == 0]
#     # your prediction code here
#     current_test['answered_correctly'] = 0.5
#     set_predict(current_test.loc[:,['row_id', 'answered_correctly']])
#     pbar.update(len(current_test))
#

In [ ]:
%%time

# pbar = tqdm(total=2500000)
pbar = tqdm(total=valid_pd.shape[0])
nlectures = valid_pd.loc[valid_pd.content_type_id == 1].shape[0]
previous_test_df = None
for (current_test, current_prediction_df) in iter_test:
    if previous_test_df is not None:
        previous_test_ar = np.array([
            eval(current_test["prior_group_answers_correct"].iloc[0]),
            eval(current_test["prior_group_responses"].iloc[0]),

        ])
        # your feature extraction and model training code here

    # curtl =  len(current_test)
    # current_list.append(curtl)
    previous_test_df = current_test.copy()
    current_test = current_test[current_test.content_type_id == 0]
    # your prediction code here
    current_test['answered_correctly'] = 0.5
    set_predict(current_test.loc[:,['row_id', 'answered_correctly']])
    pbar.update(len(current_test))


In [ ]:
curlist_df = pd.DataFrame(current_list, columns=["batch_size"])

In [ ]:
plx.histogram(current_list, nbins=10, width=200, height=300)

In [ ]:
%%time
# train_dd = train_dd.loc[:10000]
print("1 train_dd.head()")

# train_dd.head()
print(type(train_pd))
train_pd.shape


In [ ]:
%%time

if "row_id" in train_dd.columns:
    # train.set_index("row_id", inplace=True)
    train_dd = train_dd.set_index("row_id")
    # res = train_dd.mean()
    # res = delayed(train_dd.mean)()
# res_delayed = res.compute()
print("res = train_dd.mean()")
# print(res_delayed)

In [ ]:
%%time

res_delayed = delayed(train_dd.mean)()


In [ ]:
res_delayed.visualize()

In [ ]:
%%time

with ProgressBar():
    out = res_delayed.compute()

    print(f" out res_delayed = \n {out}")

In [ ]:
res_delayed.visualize()

In [ ]:
# train_dd["attempt"] = -1

print(f" train_dd.head() 2 = ")
train_dd.head()


In [ ]:
train_dd.count().compute()

In [ ]:
len(train_dd) - train_dd.count().compute()

In [ ]:
print(train_dd.shape[0].compute(), train_dd.shape[1])

In [ ]:
pd.Series(train_dd.user_id.compute()).nunique()


In [ ]:
%%time
print("max", train_dd.user_id.compute().value_counts().max())
print("mean", train_dd.user_id.compute().value_counts().mean())
print("median", train_dd.user_id.compute().value_counts().quantile(.5))

In [ ]:
%%time
max, mean, median = compute(
    train_dd.user_id.value_counts().max(),
    train_dd.user_id.value_counts().mean(),
    train_dd.user_id.value_counts().quantile(.5)
)
print(max, mean, median)

In [ ]:
def get_user_quantiles(df, plot=False):

    quantiles = []
    for q in np.arange(start=.4, stop=.8, step=.1):
        quantiles.append((q, df.user_id.value_counts().quantile(q=q).compute()))
        if plot:
            print(f"quantile {quantiles[-1]}")
            plt.plot(*quantiles[-1], marker="o",  c="w")
    return quantiles

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
train_dd.columns

In [ ]:
# total = train_dd.user_id.nunique()
groups = train_dd.groupby(by="user_id")
# total = len(groups.size())
# print("total", total)

In [ ]:
%%time
def print_count(group):
    print("print 1 from print_count")

groups_apply = groups.sum()

In [ ]:
%%time
groups_apply.compute()

In [ ]:
iter = groups


In [ ]:
%%time

content_id_sum = groups.content_id.sum()

print("content_id_sum.compute()")
content_id_sum.compute()

In [ ]:
%%time
agg = groups.aggregate(['sum', 'mean', 'max', 'min', list]).compute()

In [ ]:
agg

In [ ]:
agg.loc[115, "timestamp"]["sum"]

In [ ]:
with timer("rolling values"):
    for id, df in tqdm(groups, total=total):
        print("ok")
        # print(f"  user_id = {id}  shape = {df.shape[0]} is_monotonic {df.timestamp.is_monotonic}")
        pass
        # train_dd.loc[df.index, "attempt"] = range(1, df.shape[0]+1)

        # quantiles = get_user_quantiles(df=df)
        #
        # for quantile, val in quantiles:
        #     train_dd.loc[df.index, f"q{quantile}_None"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type=None,
        #     ).quantile(quantile)
        #
        #     train_dd.loc[df.index, f"q{quantile}_gauss"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type="gaussian",
        #     ).mean(std=3)
        #
        #     train_dd.loc[df.index, f"q{quantile}_exp"] = df.timestamp.rolling(
        #         int(val),
        #         min_periods=1,
        #         win_type="exponential",
        #     ).sum(tau=3)

# train_dd = train_dd.compute()



In [ ]:
train_shuffled = train.sample()
print("train.shape", train.shape)
train.head(20)

In [ ]:
submit_example = pd.read_csv("./input/example_sample_submission.csv", )
test = pd.read_csv("./input/example_test.csv")
lectures = pd.read_csv("./input/lectures.csv")
questions = pd.read_csv("./input/questions.csv")
print("ok")

In [ ]:
print("submit_example.shape", submit_example.shape[0])
submit_example.head()

In [ ]:
print("test.shape", test.shape)
test.head()

In [ ]:
print("lectures.shape", lectures.shape)
lectures.head()



In [ ]:
set(train.columns).symmetric_difference(set(test.columns))

In [ ]:
set(test.columns) - set(train.columns)

In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
train.columns

In [ ]:
test.columns


In [ ]:
%time

# train_dt = dt.fread("./input/train.csv")
# print("train_dt.head()")
# train_dt.head(5)

##Questions

In [ ]:
print("questions.shape", questions.shape)
questions.head()

In [ ]:
bundle_ids = questions.value_counts(subset=["bundle_id"])
print(bundle_ids)
plt.hist(x=bundle_ids, bins=10);


In [ ]:
questions.value_counts(subset=["correct_answer"])


In [ ]:
questions.value_counts(subset=["part"])



In [ ]:
questions.tags = questions.tags.astype(str).apply([str.strip, str.split])
questions.tags[0][0]

In [ ]:
# ll = train_dt.shape[0]
#
# uniques = dt.unique(train_dt["user_id"]).shape[0]
# print(ll)
# print(uniques)
# print(ll/uniques)

In [ ]:
value_counts = train.user_id.value_counts()

In [ ]:
value_counts

In [ ]:
plt.hist(value_counts);